Notebook for 
1) Checking if the SteadyCom solution for the four *E. coli* model is a steady state.
2) Generating steady state samples for the *E. coli* model. 


In [2]:
import numpy as np
import scipy.sparse as sparse
import scipy.io as sio
import cvxpy as cp
import time
import pickle


In [8]:
# Set solver to use with cvxpy. See "Choosing a solver" section here for using alternatives 
# to Gurobi: https://www.cvxpy.org/tutorial/advanced/index.html
cp_solver = 'GUROBI'

In [ ]:
# Load data for SteadyCom solution.
directory = '../SteadyCom/'
starting_biomasses = sio.loadmat(directory + '/biomasses.mat')['biomass']
starting_fluxes = sio.loadmat(directory + '/fluxes.mat')['fluxes']

# Load data for model.
directory = '../ModelFiles/FourSpecies'

# S contains the stoichiometry matrices R_{k} and R_{k}^{ex} for each species k.
S = sio.loadmat(directory + '/S.mat')['S']
I = sio.loadmat(directory + '/I.mat')['I'][0][0]
J = sio.loadmat(directory + '/J.mat')['J'][0][0]

reaction_lb = sio.loadmat(directory + '/lb.mat')['lb']
reaction_ub = sio.loadmat(directory + '/ub.mat')['ub']

# Indices of reactions and metabolites for each species, needed because 
# cobra groups all metabolites and reactions into a single model.
lumen_reactions_idx = sio.loadmat(directory + '/lumen_reactions_idx.mat')['lumen_reactions_idx'] - 1
lumen_metabolites_idx = sio.loadmat(directory + '/lumen_metabolites_idx.mat')['lumen_metabolites_idx'] - 1
lumen_reaction_names = sio.loadmat(directory + '/lumen_reactions.mat')['lumen_reactions']

Ec1_reactions_idx = sio.loadmat(directory + '/Ec1_reactions_idx.mat')['Ec1_reactions_idx'] - 1
Ec1_reaction_names = sio.loadmat(directory + '/Ec1_reactions.mat')['Ec1_reactions']
Ec1_metabolites_idx = sio.loadmat(directory + '/Ec1_metabolites_idx.mat')['Ec1_metabolites_idx'] - 1
Ec1_biomass_idx = sio.loadmat(directory + '/Ec1_biomass_idx.mat')['Ec1_biomass_idx'][0][0]-1

Ec2_reactions_idx = sio.loadmat(directory + '/Ec2_reactions_idx.mat')['Ec2_reactions_idx'] - 1
Ec2_reaction_names = sio.loadmat(directory + '/Ec2_reactions.mat')['Ec2_reactions']
Ec2_metabolites_idx = sio.loadmat(directory + '/Ec2_metabolites_idx.mat')['Ec2_metabolites_idx'] - 1
Ec2_biomass_idx = sio.loadmat(directory + '/Ec2_biomass_idx.mat')['Ec2_biomass_idx'][0][0]-1

Ec3_reactions_idx = sio.loadmat(directory + '/Ec3_reactions_idx.mat')['Ec3_reactions_idx'] - 1
Ec3_reaction_names = sio.loadmat(directory + '/Ec3_reactions.mat')['Ec3_reactions']
Ec3_metabolites_idx = sio.loadmat(directory + '/Ec3_metabolites_idx.mat')['Ec3_metabolites_idx'] - 1
Ec3_biomass_idx = sio.loadmat(directory + '/Ec3_biomass_idx.mat')['Ec3_biomass_idx'][0][0]-1

Ec4_reactions_idx = sio.loadmat(directory + '/Ec4_reactions_idx.mat')['Ec4_reactions_idx'] - 1
Ec4_reaction_names = sio.loadmat(directory + '/Ec4_reactions.mat')['Ec4_reactions']
Ec4_metabolites_idx = sio.loadmat(directory + '/Ec4_metabolites_idx.mat')['Ec4_metabolites_idx'] - 1
Ec4_biomass_idx = sio.loadmat(directory + '/Ec4_biomass_idx.mat')['Ec4_biomass_idx'][0][0]-1

I1 = len(Ec1_metabolites_idx); I2 = len(Ec2_metabolites_idx); I3 = len(Ec3_metabolites_idx); I4 = len(Ec4_metabolites_idx)
Jl = len(lumen_reactions_idx); J1 = len(Ec1_reactions_idx); J2 = len(Ec2_reactions_idx); J3 = len(Ec3_reactions_idx); J4 = len(Ec4_reactions_idx)


In [ ]:
Ec1_reaction_names = np.array([Ec1_reaction_names[i][0] for i in range(len(Ec1_reaction_names))])
Ec2_reaction_names = np.array([Ec2_reaction_names[i][0] for i in range(len(Ec2_reaction_names))])
Ec3_reaction_names = np.array([Ec3_reaction_names[i][0] for i in range(len(Ec3_reaction_names))])
Ec4_reaction_names = np.array([Ec4_reaction_names[i][0] for i in range(len(Ec4_reaction_names))])
lumen_reaction_names = np.array([lumen_reaction_names[i][0] for i in range(len(lumen_reaction_names))])


First, we'll check to see if the output from SteadyCom for this model is a steady state GNE (after properly scaling the biomasses and fluxes).


In [ ]:
# Create vectors that can be dotted with vector of reactions for each species 
# and pull out the biomass reaction.
e1 = sparse.identity(J1 + Jl).tocsr()[:, Ec1_biomass_idx]; e2 = sparse.identity(J2 + Jl).tocsr()[:, Ec2_biomass_idx]
e3 = sparse.identity(J3 + Jl).tocsr()[:, Ec3_biomass_idx]; e4 = sparse.identity(J4 + Jl).tocsr()[:, Ec4_biomass_idx]

# Maximum iterations to find equilibrium.
max_iters = 2000

# Keep track of fluxes, biomasses, and changes in fluxes as we search for an equilibrium.
x_values = np.zeros((max_iters, J)); biomass_values = np.zeros((max_iters, 4)); relative_changes = np.zeros((max_iters-1,1))

bm1 = starting_biomasses[0][0]; bm2 = starting_biomasses[1][0]; bm3 = starting_biomasses[2][0]; bm4 = starting_biomasses[3][0]
biomass_values[0,:] = [bm1, bm2, bm3, bm4]

Sl = S[lumen_metabolites_idx.flatten(), :]
Sl = Sl[:, lumen_reactions_idx.flatten()]

# Initialize fluxes for each of the k species as the flux determined by SteadyCom.
x1 = np.zeros((J1 + Jl, 1))
x1[0:J1] = starting_fluxes[Ec1_reactions_idx.flatten()]
S1 = S[lumen_metabolites_idx.flatten(), :]
S1 = S1[:, Ec1_reactions_idx.flatten()]
x1[J1:] = -Sl.T.dot(S1.dot(starting_fluxes[Ec1_reactions_idx.flatten()]))

x2 = np.zeros((J2 + Jl, 1))
x2[0:J2] = starting_fluxes[Ec2_reactions_idx.flatten()]
S2 = S[lumen_metabolites_idx.flatten(), :]
S2 = S2[:, Ec2_reactions_idx.flatten()]
x2[J2:] = -Sl.T.dot(S2.dot(starting_fluxes[Ec2_reactions_idx.flatten()]))

x3 = np.zeros((J3 + Jl, 1))
x3[0:J3] = starting_fluxes[Ec3_reactions_idx.flatten()]
S3 = S[lumen_metabolites_idx.flatten(), :]
S3 = S3[:, Ec3_reactions_idx.flatten()]
x3[J3:] = -Sl.T.dot(S3.dot(starting_fluxes[Ec3_reactions_idx.flatten()]))

x4 = np.zeros((J4 + Jl, 1))
x4[0:J4] = starting_fluxes[Ec4_reactions_idx.flatten()]
S4 = S[lumen_metabolites_idx.flatten(), :]
S4 = S4[:, Ec4_reactions_idx.flatten()]
x4[J4:] = -Sl.T.dot(S4.dot(starting_fluxes[Ec4_reactions_idx.flatten()]))

x = starting_fluxes
x_values[0,:] = x.flatten()

# Track progress for finding equilibrium.
current_iter = 0
current_change = 1e10

# Modulate regularization term.
delta_max = 1
delta_min = 1e-3
C = 5e1
B = 6
sigmoid = lambda x : 1 / (1 + np.exp(-x))
k = np.linspace(0, max_iters, max_iters)

delta_vals = delta_min + (delta_max - delta_min) * sigmoid(k/C - B)


In [ ]:
# Set death rate so that SteadyCom solution is a steady state GNE.
death_rate = np.array([[0.5]])

# Scale SteadyCom solution to match death_rate.
shift = bm1 * death_rate / e1.T.dot(x1)
bm1 = bm1 / shift[0][0]
bm2 = bm2 / shift[0][0]
bm3 = bm3 / shift[0][0]
bm4 = bm4 / shift[0][0]


In [ ]:
x1_steadycom = x1
x2_steadycom = x2
x3_steadycom = x3
x4_steadycom = x4
bm1_steadycom = bm1
bm2_steadycom = bm2
bm3_steadycom = bm3
bm4_steadycom = bm4


In [ ]:
total_start_time = time.time()
while (current_iter < max_iters) and (current_change > 1e-8):
    print('Iteration: ', current_iter)
    start_time = time.time()

    delta = delta_vals[current_iter]

    # Ec1 update.
    print('Starting Ec1 update')
    solution1 = cp.Variable((J1 + Jl, 1))
    objective = e1.T @ solution1 - 0.5 * delta * cp.quad_form(solution1 - x1, sparse.identity(J1+Jl).tocsr())
    constraints = [S[:, np.concatenate([Ec1_reactions_idx, lumen_reactions_idx]).flatten()] @ solution1 == 0,
                   solution1[0:J1] >= bm1 * reaction_lb[Ec1_reactions_idx.flatten()],
                   solution1[0:J1] <= bm1 * reaction_ub[Ec1_reactions_idx.flatten()],
                   solution1[J1:J1+Jl] + x2[J2:] + x3[J3:] + x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution1[J1:J1+Jl] + x2[J2:] + x3[J3:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob1 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        print('Trying feastol ', feastol)

        # Need to be careful when solving for two reasons:
        # 1) The tolerances could be too small for their to be a solution.
        # 2) Occassionally Gurobi runs into an ARPACK error; 
        # this error is the result of random subprocess 
        # used by Gurobi i.e. if this error is encountered,
        # solving the problem a second time usually 
        # avoids the ARPACK error.
        try:
            prob1.solve(solver = cp_solver, warm_start = True, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob1.status
            feastol = feastol * 10
            opttol = opttol * 10
        except:
            feastol = feastol * 10
            opttol = opttol * 10

        print('status: ', status)
    print('Final status for mouse problem: ', status)

    # If the species' problem is infeasible, 
    # we'll shrink the size of its current fluxes
    # by a small amount. This is done by letting 
    # the solution in this iteration for this species 
    # be all zero.
    if status != 'optimal':
        print('In status != optimal if-loop for mouse problem')
        solution1.value = np.zeros((J1+Jl,1))

    # Ec2 update.
    print('Starting Ec2 update')
    solution2 = cp.Variable((J2 + Jl, 1))
    objective = e2.T @ solution2 - 0.5 * delta * cp.quad_form(solution2 - x2, sparse.identity(J2+Jl).tocsr())
    constraints = [S[:, np.concatenate([Ec2_reactions_idx, lumen_reactions_idx]).flatten()] @ solution2 == 0,
                   solution2[0:J2] >= bm2 * reaction_lb[Ec2_reactions_idx.flatten()],
                   solution2[0:J2] <= bm2 * reaction_ub[Ec2_reactions_idx.flatten()],
                   solution2[J2:J2+Jl] + x1[J1:] + x3[J3:] + x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution2[J2:J2+Jl] + x1[J1:] + x3[J3:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob2 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        print('Trying feastol ', feastol)

        try:
            prob2.solve(solver = cp_solver, warm_start = True, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob2.status
            feastol = feastol * 10
            opttol = opttol * 10
        except:
            feastol = feastol * 10
            opttol = opttol * 10

        print('status: ', status)
    print('Final status for mouse problem: ', status)

    if status != 'optimal':
        print('In status != optimal if-loop for mouse problem')
        solution2.value = np.zeros((J2+Jl,1))

    # Ec3 update.
    print('Starting Ec3 update')
    solution3 = cp.Variable((J3 + Jl, 1))
    objective = e3.T @ solution3 - 0.5 * delta * cp.quad_form(solution3 - x3, sparse.identity(J3+Jl).tocsr())
    constraints = [S[:, np.concatenate([Ec3_reactions_idx, lumen_reactions_idx]).flatten()] @ solution3 == 0,
                   solution3[0:J3] >= bm3 * reaction_lb[Ec3_reactions_idx.flatten()],
                   solution3[0:J3] <= bm3 * reaction_ub[Ec3_reactions_idx.flatten()],
                   solution3[J3:J3+Jl] + x2[J2:] + x1[J1:] + x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution3[J3:J3+Jl] + x2[J2:] + x1[J1:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob3 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        print('Trying feastol ', feastol)

        try:
            prob3.solve(solver = cp_solver, warm_start = True, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob3.status
            feastol = feastol * 10
            opttol = opttol * 10
        except:
            feastol = feastol * 10
            opttol = opttol * 10

        print('status: ', status)
    print('Final status for mouse problem: ', status)

    if status != 'optimal':
        print('In status != optimal if-loop for mouse problem')
        solution3.value = np.zeros((J3+Jl,1))

    # Ec4 update.
    print('Starting Ec4 update')
    solution4 = cp.Variable((J4 + Jl, 1))
    objective = e4.T @ solution4 - 0.5 * delta * cp.quad_form(solution4 - x4, sparse.identity(J4+Jl).tocsr())
    constraints = [S[:, np.concatenate([Ec4_reactions_idx, lumen_reactions_idx]).flatten()] @ solution4 == 0,
                   solution4[0:J4] >= bm4 * reaction_lb[Ec4_reactions_idx.flatten()],
                   solution4[0:J4] <= bm4 * reaction_ub[Ec4_reactions_idx.flatten()],
                   solution4[J4:J4+Jl] + x2[J2:] + x3[J3:] + x1[J1:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution4[J4:J4+Jl] + x2[J2:] + x3[J3:] + x1[J1:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob4 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        print('Trying feastol ', feastol)

        try:
            prob4.solve(solver = cp.GUROB, warm_start = True, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob4.status
            feastol = feastol * 10
            opttol = opttol * 10
        except:
            feastol = feastol * 10
            opttol = opttol * 10

        print('status: ', status)
    print('Final status for mouse problem: ', status)

    if status != 'optimal':
        print('In status != optimal if-loop for mouse problem')
        solution4.value = np.zeros((J4+Jl,1))
        
    # Update fluxes and biomasses.
    x1 = x1 + (1 / (2 + np.sqrt(current_iter))) * (solution1.value[0:J1+Jl] - x1)
    x2 = x2 + (1 / (2 + np.sqrt(current_iter))) * (solution2.value[0:J2+Jl] - x2)
    x3 = x3 + (1 / (2 + np.sqrt(current_iter))) * (solution3.value[0:J3+Jl] - x3)
    x4 = x4 + (1 / (2 + np.sqrt(current_iter))) * (solution4.value[0:J4+Jl] - x4)
    
    x[Ec1_reactions_idx.flatten()] = x1[0:J1]
    x[Ec2_reactions_idx.flatten()] = x2[0:J2]
    x[Ec3_reactions_idx.flatten()] = x3[0:J3]
    x[Ec4_reactions_idx.flatten()] = x4[0:J4]
    x[lumen_reactions_idx.flatten()] = x1[J1:] + x2[J2:] + x3[J3:] + x4[J4:]
    x_values[current_iter,:] = x.flatten()

    if current_iter > 0:
        i = current_iter
        print('Relative change in fluxes: ', np.linalg.norm((x_values[i,:] - x_values[i-1,:])) / (np.linalg.norm(x_values[i-1,:]) + 1e-6))
        relative_changes[i-1] = np.linalg.norm((x_values[i,:] - x_values[i-1,:])) / (np.linalg.norm(x_values[i-1,:]) + 1e-6)
        current_change = relative_changes[i-1]

    print('Ec1 unweighted biomass reaction rate: ', e1.T.dot(x1) / bm1)
    print('Ec2 unweighted biomass reaction rate: ', e2.T.dot(x2) / bm2)
    print('Ec3 unweighted biomass reaction rate: ', e3.T.dot(x3) / bm3)
    print('Ec4 unweighted biomass reaction rate: ', e4.T.dot(x4) / bm4)
        
    current_iter = current_iter + 1
    
    print('Time for Iteration: ', time.time() - start_time, ' seconds')
    print('\n')
    print('\n')

print(time.time() - total_start_time)


In [ ]:
print(np.max(np.abs(x1 - x1_steadycom)))
print(np.max(np.abs(x2 - x2_steadycom)))
print(np.max(np.abs(x3 - x3_steadycom)))
print(np.max(np.abs(x4 - x4_steadycom)))


Above cell shows that SteadyCom solution and solution from GameCom model
initialized at the SteadyCom solution are essentially the same.


Now, we'll try out many different target steady states to find many different steady state GNEs for this problem.


In [ ]:
def initial_guess(target_bm):
    '''Generate a feasible solution for 
    problem (8) from the paper given 
    fixed biomasses target_bm. We'll also 
    require that the growth rates be at least 
    as large as the death rate.
    If no feasible solution exists, then target_bm
    cannot possibly be a steady state.'''
    solution1 = cp.Variable((J1+Jl,1))
    solution2 = cp.Variable((J2+Jl,1))
    solution3 = cp.Variable((J3+Jl,1))
    solution4 = cp.Variable((J4+Jl,1))
    
    constraints = []
    constraints.append(S[:,np.concatenate([Ec1_reactions_idx, lumen_reactions_idx]).flatten()] @ solution1 == 0)
    constraints.append(S[:,np.concatenate([Ec2_reactions_idx, lumen_reactions_idx]).flatten()] @ solution2 == 0)
    constraints.append(S[:,np.concatenate([Ec3_reactions_idx, lumen_reactions_idx]).flatten()] @ solution3 == 0)
    constraints.append(S[:,np.concatenate([Ec4_reactions_idx, lumen_reactions_idx]).flatten()] @ solution4 == 0)
    constraints.append(death_rate <= e1.T @ solution1)
    constraints.append(death_rate <= e2.T @ solution2)
    constraints.append(death_rate <= e3.T @ solution3)
    constraints.append(death_rate <= e4.T @ solution4)
    constraints.append(reaction_lb[Ec1_reactions_idx.flatten()] <= solution1[0:J1])
    constraints.append(reaction_ub[Ec1_reactions_idx.flatten()] >= solution1[0:J1])
    constraints.append(reaction_lb[Ec2_reactions_idx.flatten()] <= solution2[0:J2])
    constraints.append(reaction_ub[Ec2_reactions_idx.flatten()] >= solution2[0:J2])
    constraints.append(reaction_lb[Ec3_reactions_idx.flatten()] <= solution3[0:J3])
    constraints.append(reaction_ub[Ec3_reactions_idx.flatten()] >= solution3[0:J3])
    constraints.append(reaction_lb[Ec4_reactions_idx.flatten()] <= solution4[0:J4])
    constraints.append(reaction_ub[Ec4_reactions_idx.flatten()] >= solution4[0:J4])
    constraints.append(target_bm[0] * solution1[J1:] + target_bm[1] * solution2[J2:] + target_bm[2] * solution3[J3:] + target_bm[3] * solution4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()])
    constraints.append(target_bm[0] * solution1[J1:] + target_bm[1] * solution2[J2:] + target_bm[2] * solution3[J3:] + target_bm[3] * solution4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()])
    
    objective = 0
    prob = cp.Problem(cp.Maximize(0), constraints)
    prob.solve(solver=cp_solver)
    
    return (solution1.value, solution2.value, solution3.value, solution4.value)
    
    

In [ ]:
def check_steady_state(target_bm, x1, x2, x3, x4):
    '''Checks to see if biomasses target_bm 
    with fluxes x1, ..., x4 is a steady state GNE.'''
    solution1 = cp.Variable((J1 + Jl, 1))
    solution2 = cp.Variable((J2 + Jl, 1))
    solution3 = cp.Variable((J3 + Jl, 1))
    solution4 = cp.Variable((J4 + Jl, 1))
    
    # Ec1 problem.
    objective = e1.T @ solution1
    constraints = [S[:, np.concatenate([Ec1_reactions_idx, lumen_reactions_idx]).flatten()] @ solution1 == 0,
                   solution1[0:J1] >= reaction_lb[Ec1_reactions_idx.flatten()],
                   solution1[0:J1] <= reaction_ub[Ec1_reactions_idx.flatten()],
                   target_bm[0] * solution1[J1:] + target_bm[1] * x2[J2:] + target_bm[2] * x3[J3:] + target_bm[3] * x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   target_bm[0] * solution1[J1:] + target_bm[1] * x2[J2:] + target_bm[2] * x3[J3:] + target_bm[3] * x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob1 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        try:
            prob1.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob1.status
        except:
            feastol = feastol * 10
            opttol = opttol * 10
    
    if status != 'optimal':
        solution1.value = np.zeros((J1+Jl,1))
        
    # Ec2 problem.
    objective = e2.T @ solution2
    constraints = [S[:, np.concatenate([Ec2_reactions_idx, lumen_reactions_idx]).flatten()] @ solution2 == 0,
                   solution2[0:J2] >= reaction_lb[Ec2_reactions_idx.flatten()],
                   solution2[0:J2] <= reaction_ub[Ec2_reactions_idx.flatten()],
                   target_bm[0] * x1[J1:] + target_bm[1] * solution2[J2:] + target_bm[2] * x3[J3:] + target_bm[3] * x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   target_bm[0] * x1[J1:] + target_bm[1] * solution2[J2:] + target_bm[2] * x3[J3:] + target_bm[3] * x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob2 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        try:
            prob2.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob2.status
        except:
            feastol = feastol * 10
            opttol = opttol * 10
    
    if status != 'optimal':
        solution2.value = np.zeros((J2+Jl,1))
    
    # Ec3 problem.
    objective = e3.T @ solution3
    constraints = [S[:, np.concatenate([Ec3_reactions_idx, lumen_reactions_idx]).flatten()] @ solution3 == 0,
                   solution3[0:J3] >= reaction_lb[Ec3_reactions_idx.flatten()],
                   solution3[0:J3] <= reaction_ub[Ec3_reactions_idx.flatten()],
                   target_bm[0] * x1[J1:] + target_bm[1] * x2[J2:] + target_bm[2] * solution3[J3:] + target_bm[3] * x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   target_bm[0] * x1[J1:] + target_bm[1] * x2[J2:] + target_bm[2] * solution3[J3:] + target_bm[3] * x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob3 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        try:
            prob3.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob3.status
        except:
            feastol = feastol * 10
            opttol = opttol * 10
    
    if status != 'optimal':
        solution3.value = np.zeros((J3+Jl,1))
    
    # Ec4 problem.
    objective = e4.T @ solution4
    constraints = [S[:, np.concatenate([Ec4_reactions_idx, lumen_reactions_idx]).flatten()] @ solution4 == 0,
                   solution4[0:J4] >= reaction_lb[Ec4_reactions_idx.flatten()],
                   solution4[0:J4] <= reaction_ub[Ec4_reactions_idx.flatten()],
                   target_bm[0] * x1[J1:] + target_bm[1] * x2[J2:] + target_bm[2] * x3[J3:] + target_bm[3] * solution4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   target_bm[0] * x1[J1:] + target_bm[1] * x2[J2:] + target_bm[2] * x3[J3:] + target_bm[3] * solution4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()]]
    prob4 = cp.Problem(cp.Maximize(objective), constraints)
    feastol = 1e-8
    opttol = 1e-8
    status = 'infeasible'
    while status != 'optimal' and feastol < 1e-6:
        try:
            prob4.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
            status = prob4.status
        except:
            feastol = feastol * 10
            opttol = opttol * 10
    
    if status != 'optimal':
        solution4.value = np.zeros((J4+Jl,1))
    
    if (prob1.value - e1.T.dot(x1) < 1e-4) and (prob2.value - e2.T.dot(x2) < 1e-4) and (prob3.value - e3.T.dot(x3) < 1e-4) and (prob4.value - e4.T.dot(x4) < 1e-4):
        steady_state = True
    else:
        steady_state = False
    
    return (steady_state, prob1, solution1, prob2, solution2, prob3, solution3, prob4, solution4)


In [ ]:
def compute_steady_state(target_bm, x1, x2, x3, x4):
    ''' Use problem (11) to compute a steady state GNE,
    starting from biomasses target_bm and fluxes x1, ..., x4.
    '''
    current_iter = 0
    current_change1 = 1e10
    current_change_bm1 = 1e10
    current_change2 = 1e10
    current_change_bm2 = 1e10
    current_change3 = 1e10
    current_change_bm3 = 1e10
    current_change4 = 1e10
    current_change_bm4 = 1e10
    
    bm1 = target_bm[0]
    bm2 = target_bm[1]
    bm3 = target_bm[2]
    bm4 = target_bm[3]
    
    x1 = x1 * bm1
    x2 = x2 * bm2
    x3 = x3 * bm3
    x4 = x4 * bm4
    
    while (current_iter < max_iters) and (current_change1 > 1e-8) and (current_change2 > 1e-8) and (current_change3 > 1e-8) and (current_change4 > 1e-8) and (current_change_bm1 > 1e-8) and (current_change_bm2 > 1e-8) and (current_change_bm3 > 1e-8) and (current_change_bm4 > 1e-8):
        print('Iteration ', current_iter)
        delta = delta_vals[current_iter]
        
        solution1 = cp.Variable((J1 + Jl + 1, 1))
        solution2 = cp.Variable((J2 + Jl + 1, 1))
        solution3 = cp.Variable((J3 + Jl + 1, 1))
        solution4 = cp.Variable((J4 + Jl + 1, 1))
        
        # Ec1 problem.
        objective = solution1[-1] - 0.5 * delta * cp.quad_form(solution1[0:J1+Jl] - x1, sparse.identity((J1+Jl)).tocsr()) - 0.5 * delta * cp.power(solution1[-1] - bm1, 2)
        constraints = [S[:, np.concatenate([Ec1_reactions_idx, lumen_reactions_idx]).flatten()] @ solution1[0:J1+Jl] == 0,
                   solution1[0:J1] >= reaction_lb[Ec1_reactions_idx.flatten()] @ solution1[-1][:,None],
                   solution1[0:J1] <= reaction_ub[Ec1_reactions_idx.flatten()] @ solution1[-1][:,None],
                   solution1[J1:J1+Jl] + x2[J2:] + x3[J3:] + x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution1[J1:J1+Jl] + x2[J2:] + x3[J3:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()],
                   death_rate @ solution1[-1] <= e1.T @ solution1[0:J1+Jl]    
        ]
        prob1 = cp.Problem(cp.Maximize(objective), constraints)
        feastol = 1e-8
        opttol = 1e-8
        status = 'infeasible'
        while status != 'optimal' and feastol < 1e-6:
            try:
                prob1.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
                status = prob1.status
            except:
                feastol = feastol * 10
                opttol = opttol * 10

        if status != 'optimal':
            solution1.value = np.zeros((J1+Jl+1,1))
            
        # Ec2 problem.
        objective = solution2[-1] - 0.5 * delta * cp.quad_form(solution2[0:J2+Jl] - x2, sparse.identity((J2+Jl)).tocsr()) - 0.5 * delta * cp.power(solution2[-1] - bm2, 2)
        constraints = [S[:, np.concatenate([Ec2_reactions_idx, lumen_reactions_idx]).flatten()] @ solution2[0:J2+Jl] == 0,
                   solution2[0:J2] >= reaction_lb[Ec2_reactions_idx.flatten()] @ solution2[-1][:,None],
                   solution2[0:J2] <= reaction_ub[Ec2_reactions_idx.flatten()] @ solution2[-1][:,None],
                   solution2[J2:J2+Jl] + x1[J1:] + x3[J3:] + x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution2[J2:J2+Jl] + x1[J1:] + x3[J3:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()],
                   death_rate @ solution2[-1] <= e2.T @ solution2[0:J2+Jl]    
        ]
        prob2 = cp.Problem(cp.Maximize(objective), constraints)
        feastol = 1e-8
        opttol = 1e-8
        status = 'infeasible'
        while status != 'optimal' and feastol < 1e-6:
            try:
                prob2.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
                status = prob2.status
            except:
                feastol = feastol * 10
                opttol = opttol * 10

        if status != 'optimal':
            solution2.value = np.zeros((J2+Jl+1,1))
        
        # Ec3 problem.
        objective = solution3[-1] - 0.5 * delta * cp.quad_form(solution3[0:J3+Jl] - x3, sparse.identity((J3+Jl)).tocsr()) - 0.5 * delta * cp.power(solution3[-1] - bm3, 2)
        constraints = [S[:, np.concatenate([Ec3_reactions_idx, lumen_reactions_idx]).flatten()] @ solution3[0:J3+Jl] == 0,
                   solution3[0:J3] >= reaction_lb[Ec3_reactions_idx.flatten()] @ solution3[-1][:,None],
                   solution3[0:J3] <= reaction_ub[Ec3_reactions_idx.flatten()] @ solution3[-1][:,None],
                   solution3[J3:J3+Jl] + x1[J1:] + x2[J2:] + x4[J4:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution3[J3:J3+Jl] + x1[J1:] + x2[J2:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()],
                   death_rate @ solution3[-1] <= e3.T @ solution3[0:J3+Jl]    
        ]
        prob3 = cp.Problem(cp.Maximize(objective), constraints)
        feastol = 1e-8
        opttol = 1e-8
        status = 'infeasible'
        while status != 'optimal' and feastol < 1e-6:
            try:
                prob3.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
                status = prob3.status
            except:
                feastol = feastol * 10
                opttol = opttol * 10

        if status != 'optimal':
            solution3.value = np.zeros((J3+Jl+1,1))
        
        # Ec4 problem.
        objective = solution4[-1] - 0.5 * delta * cp.quad_form(solution4[0:J4+Jl] - x4, sparse.identity((J4+Jl)).tocsr()) - 0.5 * delta * cp.power(solution4[-1] - bm4, 2)
        constraints = [S[:, np.concatenate([Ec4_reactions_idx, lumen_reactions_idx]).flatten()] @ solution4[0:J4+Jl] == 0,
                   solution4[0:J4] >= reaction_lb[Ec4_reactions_idx.flatten()] @ solution4[-1][:,None],
                   solution4[0:J4] <= reaction_ub[Ec4_reactions_idx.flatten()] @ solution4[-1][:,None],
                   solution4[J4:J4+Jl] + x1[J1:] + x3[J3:] + x2[J2:] >= reaction_lb[lumen_reactions_idx.flatten()],
                   solution4[J4:J4+Jl] + x1[J1:] + x3[J3:] + x2[J2:] <= reaction_ub[lumen_reactions_idx.flatten()],
                   death_rate @ solution4[-1] <= e4.T @ solution4[0:J4+Jl]    
        ]
        prob4 = cp.Problem(cp.Maximize(objective), constraints)
        feastol = 1e-8
        opttol = 1e-8
        status = 'infeasible'
        while status != 'optimal' and feastol < 1e-6:
            try:
                prob4.solve(solver = cp_solver, FeasibilityTol = feastol, OptimalityTol = opttol)
                status = prob4.status
            except:
                feastol = feastol * 10
                opttol = opttol * 10

        if status != 'optimal':
            solution4.value = np.zeros((J4+Jl+1,1))
        
        # Update fluxes and biomasses.
        
        x1_old = x1
        x2_old = x2
        x3_old = x3
        x4_old = x4
        
        bm1_old = bm1
        bm2_old = bm2
        bm3_old = bm3
        bm4_old = bm4
        
        x1 = x1 + (1 / (2 + np.sqrt(current_iter))) * (solution1.value[0:J1+Jl] - x1)
        x2 = x2 + (1 / (2 + np.sqrt(current_iter))) * (solution2.value[0:J2+Jl] - x2)
        x3 = x3 + (1 / (2 + np.sqrt(current_iter))) * (solution3.value[0:J3+Jl] - x3)
        x4 = x4 + (1 / (2 + np.sqrt(current_iter))) * (solution4.value[0:J4+Jl] - x4)
        
        bm1 = bm1 + (1 / (2 + np.sqrt(current_iter))) * (solution1.value[-1] - bm1)
        bm2 = bm2 + (1 / (2 + np.sqrt(current_iter))) * (solution2.value[-1] - bm2)
        bm3 = bm3 + (1 / (2 + np.sqrt(current_iter))) * (solution3.value[-1] - bm3)
        bm4 = bm4 + (1 / (2 + np.sqrt(current_iter))) * (solution4.value[-1] - bm4)
        
        current_change1 = np.linalg.norm(x1 - x1_old) / np.linalg.norm(x1_old)
        current_change2 = np.linalg.norm(x2 - x2_old) / np.linalg.norm(x2_old)
        current_change3 = np.linalg.norm(x3 - x3_old) / np.linalg.norm(x3_old)
        current_change4 = np.linalg.norm(x4 - x4_old) / np.linalg.norm(x4_old)
        
        current_change_bm1 = np.linalg.norm(bm1 - bm1_old) / np.linalg.norm(bm1_old)
        current_change_bm2 = np.linalg.norm(bm2 - bm2_old) / np.linalg.norm(bm2_old)
        current_change_bm3 = np.linalg.norm(bm3 - bm3_old) / np.linalg.norm(bm3_old)
        current_change_bm4 = np.linalg.norm(bm4 - bm4_old) / np.linalg.norm(bm4_old)
        
        current_iter = current_iter + 1

    x1 = x1 / bm1
    x2 = x2 / bm2
    x3 = x3 / bm3
    x4 = x4 / bm4
    
    return (x1, x2, x3, x4, bm1, bm2, bm3, bm4)


In [ ]:
sample = np.random.uniform(0,1,size=(1000,4))
sample = sample / np.sum(sample, axis = 1)[:,None]

In [ ]:
start_time_total = time.time()
time_sum = 0
steady_states = []
for i in range(sample.shape[0]):
    start_time_i = time.time()
    print('i: ', i)
    target_bm = sample[i,:]
    x1, x2, x3, x4 = initial_guess(target_bm)
    if (x1 is None) or (x2 is None) or (x3 is None) or (x4 is None):
        continue
    print(sum(target_bm))
    steady_state, prob1, solution1, prob2, solution2, prob3, solution3, prob4, solution4 = check_steady_state(target_bm, x1, x2, x3, x4)
    if steady_state:
        steady_states.append((target_bm, solution1.value, solution2.value, solution3.value, solution4.value))
        end_time_i = time.time()
        time_sum = time_sum + (end_time_i - start_time_i)
    else:
        x1, x2, x3, x4, bm1, bm2, bm3, bm4 = compute_steady_state(target_bm, solution1.value, solution2.value, solution3.value, solution4.value)
        steady_state, prob1, solution1, prob2, solution2, prob3, solution3, prob4, solution4 = check_steady_state([bm1, bm2, bm3, bm4], x1, x2, x3, x4)
        steady_states.append((target_bm, solution1.value, solution2.value, solution3.value, solution4.value))
        end_time_i = time.time()
        time_sum = time_sum + (end_time_i - start_time_i)

end_time_total = time.time()


In [ ]:
steady_states.append(([bm1_steadycom, bm2_steadycom, bm3_steadycom, bm4_steadycom], 
                    x1_steadycom / bm1_steadycom, 
                    x2_steadycom / bm2_steadycom, 
                    x3_steadycom / bm3_steadycom, 
                    x4_steadycom / bm4_steadycom))


In [ ]:
pickle.dump(steady_states, open("steady_states.p", "wb"))
